# Chapter 04. 필터링

## 01. 필터링 이해하기

### 영상의 필터링(Image filtering)
- 영상에서 필요한 정보만 통과시키고 원치 않는 정보는 걸러내는 작업

### 주파수 공간에서의 필터링(Frequency domain filtering)
- FFT로 주파수 공간으로 변환, IFFT로 중간 주파수만 통과시켜 필터링

### 공간적 필터링(Spatial domain filtering)
- 영상의 픽셀값을 직접 이용하는 필터링 방법
    - 대상 좌표의 픽셀 값과 주변 픽셀 값을 동시에 사용
- 주로 실수형 행렬 형태의 **마스크(mask) 연산**을 이용함
    - 마스크 = 커널(kernel) = 윈도우(window) = 템플릿(template)

### 다양한 모양과 크기의 마스크
- 대부분 3 × 3 형태의 마스크를 사용
- 필터 정중앙의 Anchor(고정점)로 필터링 하고자 하는 대상 픽셀 위치를 나타냄
- 마스크의 형태와 값에 따라 필터의 역할이 결정됨
    - 영상 부드럽게 만들기
    - 영상 날카롭게 만들기
    - 엣지(edge) 검출
    - 잡음 제거

### 3 × 3 크기의 마스크를 이용한 공간적 필터링
- 마스크가 정해지면 입력 영상 위에 올려 둔 후, 동일한 위치끼리 곱한 후 모든 값을 더하여 출력 영상의 앵커 위치에 넣음 (Correlation 혹은 Convolution라고 함)
$$g(x, y) = \sum_{j=0}^{2}\sum_{i=0}^{2}m(i, j)f(x+i-1, y+j-1)$$

### 최외곽 픽셀 처리
- OpenCV는 최외곽 바깥에 가상의 픽셀이 있다고 가정한 후 필터링 연산을 처리
- OpenCV 필터링에서 지원하는 가장자리 픽셀 확장 방법
    - BORDER_CONSTANT: 가상의 픽셀은 모두 0으로
    - BORDER_REPLICATE: 가장자리 값이 쭉 이어지도록
    - BORDER_REFLECT: 가장자리를 포함하여 거울처럼 대칭으로
    - *BORDER_REFLECT_101*: 가장자리를 기준으로 안과 밖이 대칭으로 (= BORDER_REFLECT101, BORDER_DEFAULT)

### 기본적인 2D 필터링
```py
cv2.filter2D(src, ddepth, kernel, dst=None, anchor=None, delta=None, borderType=None) -> dst
```
- src: 입력 영상
- ddepth: 출력 영상 데이터 타입(e.g cv2.CV_8U, cv2.CV_32F, cv2.CV_64F). -1을 지정하면 src와 같은 타입의 dst 영상을 생성
- kernel: 필터 마스크 행렬. 실수형.
- anchor: 고정점 위치. (-1, -1)이면 필터 중앙을 고정점으로 사용
- delta: 추가적으로 더할 값
- borderType: 가장자리 픽셀 확장 방식
- dst: 출력 영상

## 02. 블러링(1): 평균값 필터

### 평균값 필터(Mean filter)
- 영상의 특정 좌표 값을 주변 픽셀 값들의 산술 평균으로 설정
- 픽셀들 간의 그레이스케일 값 변화가 줄어들어 날카로운 에지가 무뎌지고, 영상에 있는 잡음의 영향이 사라지는 효과
- 마스크 크기가 커질수록 평균값 필터 결과가 더욱 부드러워짐  
  → 더 많은 연산량이 필요!

### filter2D() 함수를 이용한 평균값 필터링 예제

In [4]:
import sys, cv2
import numpy as np

src = cv2.imread('./data/images/rose.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# kernel = np.array([
#     [1/9, 1/9, 1/9],
#     [1/9, 1/9, 1/9],
#     [1/9, 1/9, 1/9]], dtype=np.float64)
kernel = np.ones((3, 3), dtype=np.float64) / 9

dst = cv2.filter2D(src, -1, kernel)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

### 평균값 필터링 함수
```py
cv2.blur(src, ksize, dst=None, anchor=None, borderType=None) -> ds
```
- src: 입력 영상
- ksize: 평균값 필터 크기. (width, height) 형태의 튜플.
- dst: 결과 영상. 입력 영상과 같은 크기 & 같은 타입.
$$kernel = \frac{1}{ksize.width×ksize.height}\begin{bmatrix}1&1&\cdots&1\\1&1&\cdots&1\\\vdots&\vdots&\ddots&\vdots\\1&1&\cdots&1\end{bmatrix}$$

### 다양한 크기의 커널을 사용한 평균값 필터링 예제

In [7]:
import sys, cv2
import numpy as np

src = cv2.imread('./data/images/rose.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

cv2.imshow('src', src)

for ksize in (3, 5, 7):
    dst = cv2.blur(src, ksize=(ksize, ksize))

    desc = f'Mean: {ksize}x{ksize}'
    cv2.putText(dst, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)

    cv2.imshow('dst', dst)
    cv2.waitKey()

cv2.destroyAllWindows()

## 03. 블러링(2): 가우시안 필터

### 평균값 필터에 의한 블러링의 단점
- 필터링 대상 위치에서 가까이 있는 픽셀과 멀리 있는 픽셀이 모두 같은 가중치를 사용하여 평균을 계산
- 멀리 있는 픽셀의 영향을 많이 받을 수 있음  
  → 가우시안 필터의 경우 가까운 픽셀은 큰 가중치를, 멀리 있는 픽셀은 작은 가중치를 사용하여 평균 계산

### (1차원) 가우시안 함수 (Gaussian function, 정규분포)
$$ G_{μ, σ}(x) = \frac{1}{\sqrt{2π}σ}e^{-\frac{(x-μ)^2}{2σ^2}} $$
- μ: 평균
- σ: 표준편차
- σ$^2$: 분산

### 가우시안 함수의 특징
- Symmetric (bell curve) shape around the mean: 좌우 대칭, 기준은 평균
- mean = median = mode: 평균 = 중앙값 = 최빈값
- area under the curve = 1: 그래프의 면적은 항상 1
- 68%(-1~1σ) : 95%(-2~2σ) : 99.7%(-3~3σ)

### 2차원 가우시안 함수 ($ μ_x=μ_y=0, σ_x=σ_y=σ $)
$$ G_{σ}(x, y) = \frac{1}{2πσ^2}e^{-\frac{(x^2+y^2)^2}{2σ^2}}, \begin{cases}μ_x=μ_y=0\\σ_x=σ_y=σ\end{cases} $$

### 2차원 가우시안 필터 마스크 (σ = 1.0)
- 필터 마스크 크기: **(8σ + 1)** 또는 **(6σ + 1)**

### 가우시안 필터링 함수
```py
cv2.GaussianBlur(src, ksize, sigmaX, dst=None, sigmaY=None, borderType=None) -> dst
```
- src: 입력 영상. 각 채널별로 처리됨.
- dst: 출력 영상. src와 같은 크기, 같은 타입.
- ksize: 가우시안 커널 크기. **(0, 0)** 을 지정하면 sigma 값에 의해 자동 결정됨.
- sigmaX: x방향 sigma.
- sigmaY: y방향 sigma. 0이면 sigmaX와 같게 설정.
- borderType: 가장자리 픽셀 확장 방식.

### 다양한 크기의 sigma를 사용한 가우시안 필터링

In [10]:
import cv2

src = cv2.imread('./data/images/rose.bmp', cv2.IMREAD_GRAYSCALE)

cv2.imshow('src', src)

for sigma in range(1, 6):
    dst1 = cv2.GaussianBlur(src, (0, 0), sigma)
    dst2 = cv2.blur(src, (7, 7))
    
    desc = f'sigma = {sigma}'
    cv2.putText(dst1, desc, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 1, cv2.LINE_AA)
    
    cv2.imshow('Gaussian Filter', dst1)
    cv2.imshow('Mean Filter', dst2)
    cv2.waitKey()

cv2.destroyAllWindows()

## 04. 샤프닝: 언샤프 마스크 필터

### 언샤프 마스크(Unsharp mask) 필터링
- 날카롭지 않은(unsharp) 영상, 즉, 부드러워진 영상을 이용하여 날카로운 영상을 생성
- g(x): 원래 영상에서 부드러워진 영상의 픽셀을 뺀, 영상에서 순수하게 얻을 수 있는 날카로운 성분
$$g(x) = f(x) - \bar f(x)$$
- h(x): 원래 영상보다 대비(Contrast)가 높아진 날카로운 영상
$$h(x) = f(x) + g(x) = 2f(x) - \bar f(x)$$

### 언샤프 마스크 필터 구현하기

In [27]:
import sys, cv2
import numpy as np

src = cv2.imread('./data/images/rose.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

blr = cv2.GaussianBlur(src, (0, 0), 2.0)
# dst = cv2.addWeighted(src, 2, blr, -1, 0)
dst = np.clip(2.0 * src - blr, 0, 255).astype(np.uint8)

cv2.imshow('src', src)
cv2.imshow('blr', blr)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

#### 샤프닝 정도를 조절할 수 있도록 수식 변경
$$ h(x, y) = f(x, y) + \alpha · g(x, y) $$
$$ h(x, y) = f(x, y) + \alpha(f(x, y)-\bar f(x, y)) = (1+\alpha)f(x, y)- \alpha · \bar f(x, y)$$
$$ h(x, y) = (1 + \alpha)f(x, y)-\alpha·G_σ(f(x, y)) $$

### 컬러 영상에 대한 언샤프 마스크 필터 구현하기

In [28]:
import sys, cv2
import numpy as np

src = cv2.imread('./data/images/rose.bmp')

if src is None:
    print('Image load failed!')
    sys.exit()

src_ycrcb = cv2.cvtColor(src, cv2.COLOR_BGR2YCrCb)

src_f = src_ycrcb[:, :, 0].astype(np.float32)
blr = cv2.GaussianBlur(src_f, (0, 0), 2.0)
src_ycrcb[:, :, 0] = np.clip(2. * src_f - blr, 0, 255).astype(np.uint8)

dst = cv2.cvtColor(src_ycrcb, cv2.COLOR_YCrCb2BGR)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

## 05. 잡음 제거(1): 미디언 필터

### 영상의 잡음(Noise)
- 영상의 픽셀 값에 추가되는 원치 않는 형태의 신호
    - f: 획득된 영상, s: 원본 신호, n: 잡음
$$ f(x, y) = s(x, y) + n(x, y) $$

#### 잡음의 종류
- 가우시안 잡음 (Gaussian noise)
- 소금 & 후추 잡음 (Salt & Pepper)

### 미디언 필터(Median filter)
- 주변 픽셀들의 값들을 정렬하여 그 중앙값(median)으로 픽셀 값을 대체
- 소금-후추 잡음 제거에 효과적

### 미디언 필터링 함수
```py
cv2.medianBlur(src, ksize, dst=None) -> dst
```
- src: 입력 영상. 각 채널별로 처리됨.
- ksize: 커널 크기. 1보다 큰 홀수를 지정. 
- dst: 출력 영상. src와 같은 크기, 같은 타입.

### 미디언 필터링 예제

In [31]:
import sys, cv2

src = cv2.imread('./data/images/noise.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

dst = cv2.medianBlur(src, 3)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

## 06. 잡음 제거(2): 양방향 필터

### 가우시안 필터
- 가우시안 잡음 제거에는 가우시안 필터가 효과적

### 양방향 필터(Bilateral filter)
- 에지 보전 잡음 제거 필터(edge-preserving noise removal filter)의 하나
- 평균값 필터 또는 가우시안 필터는 에지 부근에서도 픽셀 값을 평탄하게 만드는 단점이 있음
- 기준 픽셀과 이웃 픽셀과의 거리, 그리고 픽셀 값의 차이를 함께 고려하여 블러링 정도를 조절
$$ BF[I]_p = \frac{1}{W_p} \sum_{q\in S}G_{σ_s} (||p-q||)G_{σ_r}(|I_p-I_q|)I_q $$

#### (일반적인) 가우시안 필터링: 영상 전체에서 blurring
- Same Gaussian kernel everywhere.
#### 양방향 필터: 에지가 아닌 부분에서만 blurring
- The kernel shape depends on the image content.

### 양방향 필터링 함수
```py
cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace, dst=None, borderType=None) -> dst
```
- src: 입력 영상. 8비트 또는 실수형, 1채널 또는 3채널.
- d: 필터링에 사용될 이웃 픽셀의 거리(지름). 음수(-1)를 입력하면 sigmaSpace 값에 의해 자동 결정됨.
- sigmaColor: 색 공간에서 필터의 표준 편차
- sigmaSpace: 좌표 공간에서 필터의 표준 편차
- dst: 출력 영상. src와 같은 크기, 같은 타입.
- borderType: 가장자리 픽셀 처리 방식

### 양방향 필터링 예제

In [34]:
import sys, cv2
import numpy as np

src = cv2.imread('./data/images/lenna.bmp')

if src is None:
    print('Image load failed!')
    sys.exit()

dst = cv2.bilateralFilter(src, -1, 10, 5)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

## 07. 실전 코딩: 카툰 필터 카메라

### 카툰 필터 카메라
- 카메라 입력 영상에 실시간으로 재미있는 필터링을 적용하는 기능

### 구현할 기능
- 카툰 필터
- 스케치 필터
- 스페이스바를 누를 때마다 모드 변경

#### 카툰 필터
- 입력 영상의 색상을 단순화시키고, 에지 부분을 검정색으로 강조
    - cv2.bilateralFilter(), 255 - cv2.Canny(), cv2.bitwise_and()

#### 스케치 필터
- 평탄한 영역은 흰색
- 에지 근방에서 어두운 영역을 검정색으로 설정. (밝은 영역은 흰색)
- cv2.cvtColor(), cv2.GaussianBlur(), cv2.divide() * 255

In [40]:
import sys
import cv2


def cartoon_filter(img):
    # 단순화를 효과적으로 하기 위해 입력 연산을 작게 만들어 처리 후, 최종 결과 부분에서 확대.
    h, w = img.shape[:2]
    img2 = cv2.resize(img, (w//2, h//2))

    blr = cv2.bilateralFilter(img2, -1, 20, 7)
    edge = 255 - cv2.Canny(img2, 80, 120)
    edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

    dst = cv2.bitwise_and(blr, edge)
    dst = cv2.resize(dst, (w, h), interpolation=cv2.INTER_NEAREST)

    return dst


def pencil_sketch(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blr = cv2.GaussianBlur(gray, (0, 0), 3)
    dst = cv2.divide(gray, blr, scale=255)
    dst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)

    return dst


cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

cam_mode = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    if cam_mode == 1:
        frame = cartoon_filter(frame)
    elif cam_mode == 2:
        frame = pencil_sketch(frame)

    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)

    if key == 27:
        break
    elif key == ord(' '):
        cam_mode += 1
        if cam_mode == 3:
            cam_mode = 0

cap.release()
cv2.destroyAllWindows()